# DeepLabv3+
This notebook trains and evaluates a DeepLabv3+ model for bladder neck dissection segmentation.



### 0. Access Google Drive on Google Colab
Mount Google Drive to access the dataset stored in your Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---

### 1. Import Libraries

In [ ]:
!pip install labelme tensorflow matplotlib numpy opencv-python scikit-learn
!pip install --upgrade tensorflow # upgrade tensorflow

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Input  # Removed problematic imports
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation# Import them directly from layers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Resizing
import tensorflow_datasets as tfds
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
from sklearn.utils import shuffle
import json
from PIL import Image
import cv2
import random
from IPython.display import clear_output
import time
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Image and Mask Loader

This utility class loads surgical images and their corresponding segmentation masks
(from LabelMe JSON annotations).  

In [ ]:
class ImageMaskLoader:
    def __init__(self, data_root, folders, labels, resize_height=256, resize_width=256, background_label=0):
        self.data_root = data_root
        self.folders = folders
        self.labels = labels
        self.resize_height = resize_height
        self.resize_width = resize_width
        self.background_label = background_label

    def load_and_process_data(self):
        images = []
        masks = []

        for folder in self.folders:
            folder_path = os.path.join(self.data_root, folder)
            for file in os.listdir(folder_path):
                if file.endswith(".jpg"):
                    image_path = os.path.join(folder_path, file)
                    json_path = image_path.replace(".jpg", ".json")

                    image = cv2.imread(image_path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    original_height, original_width = image.shape[:2]
                    image_resized = cv2.resize(image, (self.resize_width, self.resize_height))

                    if not os.path.exists(json_path):
                        print(f"{file} has no corresponding JSON file. Using background mask.")
                        mask = np.full((self.resize_height, self.resize_width), self.background_label, dtype=np.uint8)
                    else:
                        with open(json_path) as f:
                            data = json.load(f)
                            mask = np.full((self.resize_height, self.resize_width), self.background_label, dtype=np.uint8)

                            for shape in data['shapes']:
                                label = shape['label']
                                if label in self.labels:
                                    points = np.array(shape['points'], dtype=np.int32)
                                    points[:, 0] = (points[:, 0] * self.resize_width / original_width).astype(int)
                                    points[:, 1] = (points[:, 1] * self.resize_height / original_height).astype(int)
                                    cv2.fillPoly(mask, [points], self.labels[label])

                    images.append(img_to_array(image_resized))
                    masks.append(mask)

        images = np.array(images, dtype="float32") / 255.0
        masks = np.array(masks, dtype="int32")
        masks = np.expand_dims(masks, axis=-1)
        masks = to_categorical(masks, num_classes=len(self.labels))

        return images, masks

- **DATA_ROOT**: Root directory of the dataset on Google Drive.  
- **FOLDERS**: List of subfolders for training data.  

In [ ]:
DATA_ROOT = '/content/drive/MyDrive/your_path'

FOLDERS   = []

LABELS = {
    "background": 0,
    "bladder": 1,
    "border": 2,
    "catheter": 3,
    "gauze": 4,
    "instrument": 5,
    "mucosa": 6,
    "prostate": 7,
    "suction": 8,
    "urethra": 9,
}

In [ ]:
loader = ImageMaskLoader(DATA_ROOT, FOLDERS, LABELS)
images, masks = loader.load_and_process_data()

### Train / Test Split

In [ ]:
train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

### IoU Calculator
This utility class provides functions to calculate Intersection over Union (IoU).

In [ ]:
class IoUCalculator:
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def calculate_iou(self, y_true, y_pred):
        iou_metric = MeanIoU(num_classes=self.num_classes)
        iou_metric.update_state(y_true, y_pred)
        return iou_metric.result().numpy()

    def calculate_class_iou(self, test_masks, y_pred_argmax, class_index):
        y_true_class = (np.argmax(test_masks, axis=-1) == class_index).astype(np.uint8)
        y_pred_class = (y_pred_argmax == class_index).astype(np.uint8)
        iou = self.calculate_iou(y_true_class, y_pred_class)
        return iou

    def calculate_mean_iou(self, test_masks, y_pred_argmax):
        class_ious = []
        for cls_idx in range(self.num_classes):
            iou = self.calculate_class_iou(test_masks, y_pred_argmax, cls_idx)
            class_ious.append(iou)
        mean_iou = np.nanmean(class_ious)
        return mean_iou

### DeepLabv3+ Model Definition
This block defines the DeepLabv3+ architecture with an Xception backbone.

In [ ]:
# DeepLabv3+
def global_average_pooling_branch(input_tensor, output_channels):
    gap = GlobalAveragePooling2D()(input_tensor)
    gap = Reshape((1, 1, input_tensor.shape[-1]))(gap)
    gap = Conv2D(output_channels, (1, 1), padding='same', use_bias=False)(gap)
    gap = BatchNormalization()(gap)
    gap = Activation('relu')(gap)
    gap = UpSampling2D(size=(input_tensor.shape[1], input_tensor.shape[2]), interpolation='bilinear')(gap)
    return gap


def atrous_spatial_pyramid_pooling(x):
    b0 = Conv2D(256, (1, 1), padding='same', use_bias=False)(x)
    b0 = BatchNormalization()(b0)
    b0 = Activation('relu')(b0)

    b1 = Conv2D(256, (3, 3), padding='same', dilation_rate=6, use_bias=False)(x)
    b1 = BatchNormalization()(b1)
    b1 = Activation('relu')(b1)

    b2 = Conv2D(256, (3, 3), padding='same', dilation_rate=12, use_bias=False)(x)
    b2 = BatchNormalization()(b2)
    b2 = Activation('relu')(b2)

    b3 = Conv2D(256, (3, 3), padding='same', dilation_rate=18, use_bias=False)(x)
    b3 = BatchNormalization()(b3)
    b3 = Activation('relu')(b3)
    b4 = global_average_pooling_branch(x, 256)

    x = Concatenate()([b0, b1, b2, b3, b4])
    return x


def Deeplabv3Plus(input_shape=(256, 256, 3), num_classes=10):
    inputs = Input(shape=input_shape)

    # backbone(Xception)
    base_model = Xception(weights='imagenet', include_top=False, input_tensor=inputs)

    # ASPP
    x = base_model.get_layer('block13_sepconv2_bn').output
    x = atrous_spatial_pyramid_pooling(x)

    # decoder
    skip = base_model.get_layer('block3_sepconv2_bn').output
    skip = Resizing(height=64, width=64, interpolation='bilinear')(skip)
    skip = Conv2D(48, (1, 1), padding='same', use_bias=False)(skip)
    skip = BatchNormalization()(skip)
    skip = Activation('relu')(skip)

    x = UpSampling2D(size=(4, 4), interpolation='bilinear')(x)
    x = Concatenate()([x, skip])

    x = Conv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # output layer
    x = UpSampling2D(size=(4, 4), interpolation='bilinear')(x)
    outputs = Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# model
model = Deeplabv3Plus(input_shape=(256, 256, 3), num_classes=10)
model.summary()

### Training Configuration

- **Learning Rate Scheduler**  

- **Early Stopping**  

- **Optimizer**  

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.99

lr_scheduler = LearningRateScheduler(scheduler)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

### Model Construction and Compilation

  - Loss: Categorical Cross-Entropy

In [ ]:
input_shape = (256, 256, 3)
num_classes = len(labels)

model = Deeplabv3Plus(input_shape=input_shape, num_classes=num_classes)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


### Model Training, Monitoring, and Saving

In [ ]:
start_time = time.time()

history = model.fit(
    train_images,
    train_masks,
    validation_data=(test_images, test_masks),
    batch_size=8,
    epochs=50,
    callbacks=[lr_scheduler, early_stopping]
)


end_time = time.time()

training_time = end_time - start_time

print(f"Training time: {training_time:.2f} seconds")

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

model_path = '/content/drive/MyDrive/your_path/DLv3p_CrossEntropy.h5'
model.save(model_path)

### Model Evaluation (IoU Calculation)

In [ ]:
num_classes =  len(labels)
iou_calculator = IoUCalculator(num_classes)
y_pred = model.predict(test_images)
y_pred_argmax = np.argmax(y_pred, axis=-1)
iou_bladder = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=1)
iou_prostate = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=7)
iou_border = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=2)
iou_catheter = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=3)
iou_gauze = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=4)
iou_instrument = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=5)
iou_mucosa = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=6)
iou_suction = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=8)
iou_urethra = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=9)
iou_background = iou_calculator.calculate_class_iou(test_masks, y_pred_argmax, class_index=0)
mean_iou = iou_calculator.calculate_mean_iou(test_masks, y_pred_argmax)
print(f"Mean IoU: {mean_iou}")
print(f"IoU for Bladder: {iou_bladder}")
print(f"IoU for Prostate: {iou_prostate}")
print(f"IoU for Border: {iou_border}")
print(f"IoU for Catheter: {iou_catheter}")
print(f"IoU for Gauze: {iou_gauze}")
print(f"IoU for Instrument: {iou_instrument}")
print(f"IoU for Mucosa: {iou_mucosa}")
print(f"IoU for Suction: {iou_suction}")
print(f"IoU for Urethra: {iou_urethra}")
print(f"IoU for Background: {iou_background}")